In [1]:

import numpy as np # linear algebra
import pandas as pd

In [2]:


data = pd.read_json(r'Sarcasm_Headlines_Dataset.json', lines = True)

data_v2 = pd.read_json(r'Sarcasm_Headlines_Dataset_v2.json', lines = True)



In [3]:
print(data.shape)
print(data_v2.shape)

(26709, 3)
(28619, 3)


In [4]:
final_data = pd.concat([data,data_v2] , ignore_index = True , axis =0)
print(final_data.shape)

(55328, 3)


In [5]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55328 entries, 0 to 55327
Data columns (total 3 columns):
article_link    55328 non-null object
headline        55328 non-null object
is_sarcastic    55328 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [6]:
df = pd.DataFrame()
df['text'] = final_data['headline']
df['label'] = final_data['is_sarcastic']

In [10]:


from simpletransformers.classification import ClassificationModel

model = ClassificationModel('bert', 'bert-base-cased', num_labels=2,
args={'reprocess_input_data': True, 'overwrite_output_dir': True},use_cuda=False)



Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [11]:
from sklearn.model_selection import train_test_split

df_sample = df.sample(frac = 1)

train_df,eval_df = train_test_split(df_sample,test_size = 0.3)

print(train_df.shape)
print(eval_df.shape)

(38729, 2)
(16599, 2)


In [12]:
train_df2 = pd.DataFrame({
    'text': train_df['text'].replace(r'\n', ' ', regex=True),
    'label': train_df['label']
})

eval_df2 = pd.DataFrame({
    'text': eval_df['text'].replace(r'\n', ' ', regex=True),
    'label': eval_df['label']
})

In [14]:


model.train_model(train_df2)



C:\Users\OEM\Anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:391: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/38729 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/4842 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df2)

In [ ]:
print(result)
print(model_outputs)

In [ ]:
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eval_df2['label'].tolist()
predicted = lst

In [ ]:
import sklearn
print(sklearn.metrics.classification_report(true,predicted,target_names=['No Sarcasm' ,'Sarcasm']))

In [ ]:
sklearn.metrics.accuracy_score(true,predicted)

In [ ]:
def get_result(statement):
    result = model.predict([statement])
    pos = np.where(result[1][0] == np.amax(result[1][0]))
    pos = int(pos[0])
    sentiment_dict = {0:'No Sarcasm',1:'Sarcasm'}
    print(sentiment_dict[pos])
    return

In [ ]:
get_result("Planning to get some loan for my Harvard law degree with low interest")